---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/08_BlackScholes.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:

import plotly
from IPython.display import display, HTML

plotly.offline.init_notebook_mode(connected=True)
display(
    HTML(
        '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

In [ ]:
import numpy as np
from scipy.stats import norm

def black_scholes_call(S, K, r, sigma, q, T):
    """
    Inputs:
    S = initial stock price
    K = strike price
    r = risk-free rate
    sigma = volatility
    q = dividend yield
    T = time to maturity
    """
    if sigma <= 0 or T <= 0:
        return max(0, np.exp(-q * T) * S - np.exp(-r * T) * K)
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)
    return np.exp(-q * T) * S * N1 - np.exp(-r * T) * K * N2

def black_scholes_put(S, K, r, sigma, q, T):
    """
    Inputs:
    S = initial stock price
    K = strike price
    r = risk-free rate
    sigma = volatility
    q = dividend yield
    T = time to maturity
    """
    if sigma <= 0 or T <= 0:
        return max(0, np.exp(-r * T) * K - np.exp(-q * T) * S)

    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    N1 = norm.cdf(-d1)
    N2 = norm.cdf(-d2)
    return np.exp(-r * T) * K * N2 - np.exp(-q * T) * S * N1

In [ ]:
import numpy as np
from scipy.stats import norm

def d1(S, K, r, sigma, T):
    """
    Computes d1 used in Black-Scholes.
    """
    return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def d2(S, K, r, sigma, T):
    """
    Computes d2 used in Black-Scholes.
    """
    return d1(S, K, r, sigma, T) - sigma * np.sqrt(T)

def call_price(S, K, r, sigma, T):
    """
    Black-Scholes price for a call option.
    """
    d_1 = d1(S, K, r, sigma, T)
    d_2 = d2(S, K, r, sigma, T)
    return S * norm.cdf(d_1) - K * np.exp(-r * T) * norm.cdf(d_2)

def put_price(S, K, r, sigma, T):
    """
    Black-Scholes price for a put option.
    """
    d_1 = d1(S, K, r, sigma, T)
    d_2 = d2(S, K, r, sigma, T)
    return K * np.exp(-r * T) * norm.cdf(-d_2) - S * norm.cdf(-d_1)

def greeks(S, K, r, sigma, T, option_type="call"):
    """
    Return the main Greeks for Black-Scholes.

    Parameters
    ----------
    S : float or numpy array
        Underlying price.
    K : float
        Strike price.
    r : float
        Risk-free interest rate.
    sigma : float
        Volatility.
    T : float
        Time to maturity (in years).
    option_type : str
        'call' or 'put'.
    """
    d_1 = d1(S, K, r, sigma, T)
    d_2 = d2(S, K, r, sigma, T)
    pdf_d1 = norm.pdf(d_1)
    cdf_d1 = norm.cdf(d_1)
    cdf_d2 = norm.cdf(d_2)
    
    if option_type == "call":
        # Delta
        delta = cdf_d1
        # Gamma (same for call & put)
        gamma = pdf_d1 / (S * sigma * np.sqrt(T))
        # Vega (same for call & put, but typically scaled by 0.01 if desired in %)
        vega = S * pdf_d1 * np.sqrt(T)
        # Theta
        theta = - (S * pdf_d1 * sigma) / (2 * np.sqrt(T)) \
                - r * K * np.exp(-r * T) * cdf_d2
        # Rho
        rho = K * T * np.exp(-r * T) * cdf_d2
    else:  # put
        # Delta
        delta = cdf_d1 - 1
        # Gamma (same for call & put)
        gamma = pdf_d1 / (S * sigma * np.sqrt(T))
        # Vega (same for call & put)
        vega = S * pdf_d1 * np.sqrt(T)
        # Theta
        theta = - (S * pdf_d1 * sigma) / (2 * np.sqrt(T)) \
                + r * K * np.exp(-r * T) * norm.cdf(-d_2)
        # Rho
        rho = - K * T * np.exp(-r * T) * norm.cdf(-d_2)

    return delta, gamma, vega, theta, rho